### Skill label prediction with image-based features (text and color with descriptive tags)
### Text recognition category

In [4]:
import pandas as pd 
import random
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)

# for LSTM (keras with tf backend)
import gzip
import os
import pickle
import requests
import time
import re
# os.environ['KERAS_BACKEND']='cntk'
os.environ['KERAS_BACKEND']='tensorflow'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.initializers import he_normal
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import time 
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/edithzeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/edithzeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edithzeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17755839873414014374
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 271777792
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1954441188433325375
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 365b:00:00.0, compute capability: 3.7"
]


In [5]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

## Preprocessing

In [ ]:
vizwiz_features_train_color = pd.read_csv('azure_features_images/data/vizwiz_train_color_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                    quotechar='"', error_bad_lines=False)
vizwiz_features_train_text = pd.read_csv('azure_features_images/data/vizwiz_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_features_val_color = pd.read_csv('azure_features_images/data/vizwiz_val_color_recognition.csv',
                                  delimiter=';', engine='python',
                                  dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                  quotechar='"', error_bad_lines=False)
vizwiz_features_val_text = pd.read_csv('azure_features_images/data/vizwiz_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vqa_features_train_color = pd.read_csv('azure_features_images/data/vqa_train_color_recognition.csv',
                                 delimiter=';', engine='python', 
                                 dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                 quotechar='"', error_bad_lines=False)
vqa_features_train_text = pd.read_csv('azure_features_images/data/vqa_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)
vqa_features_val_color = pd.read_csv('azure_features_images/data/vqa_val_color_recognition.csv',
                               delimiter=';', engine='python',
                               dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                               quotechar='"', error_bad_lines=False)
vqa_features_val_text = pd.read_csv('azure_features_images/data/vqa_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_targets_train = pd.read_csv('../vizwiz_skill_typ_train.csv', dtype={'QID':str},
                                   delimiter=',', quotechar='"',
                                   engine='python', error_bad_lines=False)
vizwiz_targets_val = pd.read_csv('../vizwiz_skill_typ_val.csv', dtype={'QID':str},
                                 delimiter=',', quotechar='"', engine='python', error_bad_lines=False)
vqa_targets_train = pd.read_csv('../vqa_skill_typ_train.csv', dtype={'QID':str},
                               engine='python', quotechar='"', error_bad_lines=False)
vqa_targets_val = pd.read_csv('../vqa_skill_typ_val.csv', dtype={'QID':str},
                               engine='python', quotechar='"', error_bad_lines=False)

In [51]:
vizwiz_features_train_text.head(2)

,qid,question,ocr_text,handwritten_text
0,VizWiz_train_000000000000.jpg,What's the name of this product?,"['b', 'sil', 'leaves', '0.62', 'oz', '(170)']",['NET WT O. 62 02 ( 179)']
1,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,[],['^TAKE Three 1^']


In [7]:
def join_feature_target(feature_df_text, feature_df_color, target_df):
    feature_text = copy.deepcopy(feature_df_text)
    feature_color = copy.deepcopy(feature_df_color)
    target = copy.deepcopy(target_df)
    # text features 
    feature_text.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_text.set_index('QID', inplace=True)
    # color features
    feature_color.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_color.set_index('QID', inplace=True)
    # join features
    features = feature_text.join(feature_color[['descriptions','tags','dominant_colors']],
                                 on='QID',
                                 how='outer')
    # join features with target
    target = target[['QID', 'IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH']]
    target.set_index('QID', inplace=True)
    target = target.astype(dtype=str)
    df = target.join(features, on='QID', how='inner')
    df['descriptions'].astype(list)
    return df

def lem(s):
    arr = s.split(" ")
    lem = WordNetLemmatizer()
    op = ""
    for w in arr:
        word = lem.lemmatize(w) + ' '
        op += word
    return op

def preprocess_text(feature_columns):
    """ output an nparray with single document per data point """
    ip = copy.deepcopy(feature_columns).values
    op = []
    for i in range(ip.shape[0]):
        doc      =  ""
        for j in range(ip.shape[1]):
            # clean up chars
            s    =  str(ip[i][j])
            s    =  s.translate({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+'"}).lower() + " "
            if j == 1:             # clean descriptions
                s = re.sub(r'confidence\s+\d+', '', s)
                s = re.sub(r'text', '', s)
            # lexicon normalize
            s    = lem(s)
            doc  += s
        op.append(doc)
    op = np.asarray(op)
    return op

#def remove_stop_words(features_array):
#    stop_words = set(stopwords.words(features_array))

In [8]:
vizwiz_train   = join_feature_target(vizwiz_features_train_text, 
                                   vizwiz_features_train_color, 
                                   vizwiz_targets_train)
vizwiz_val     = join_feature_target(vizwiz_features_val_text, 
                                 vizwiz_features_val_color, 
                                 vizwiz_targets_val)
vqa_train      = join_feature_target(vqa_features_train_text, 
                                   vqa_features_train_color, 
                                   vqa_targets_train)
vqa_val        = join_feature_target(vizwiz_features_val_text, 
                                 vqa_features_val_color, 
                                 vqa_targets_val)
print(vizwiz_train.shape, vqa_train.shape, vizwiz_train.shape[0] + vqa_train.shape[0])
print(vizwiz_val.shape, vqa_val.shape, vizwiz_val.shape[0] + vqa_val.shape[0])

(14257, 13) (3230, 13) 17487
(2247, 13) (513, 13) 2760


In [9]:
# VizWiz only
train = vizwiz_train
val = vizwiz_val
print("Training: {}\nValidation: {}".format(train.shape, val.shape))

features_train = preprocess_text(train[['QSN','descriptions', 'tags', 'dominant_colors', 
                                        'handwritten_text', 'ocr_text']])
txt_train      = train['TXT'].values
col_train      = train['COL'].values
cnt_train      = train['CNT'].values

features_val   = preprocess_text(val[['QSN', 'descriptions', 'tags', 'dominant_colors',
                                      'handwritten_text', 'ocr_text']])
txt_val        = val['TXT'].values.astype('float32')
col_val        = val['COL'].values.astype('float32')
cnt_val        = val['CNT'].values.astype('float32')

Training: (14257, 13)
Validation: (2247, 13)


In [32]:
# VQA only
train = vqa_train
val = vqa_val
print("Training: {}\nValidation: {}".format(train.shape, val.shape))

features_train = preprocess_text(train[['QSN','descriptions', 'tags', 'dominant_colors', 
                                        'handwritten_text', 'ocr_text']])
txt_train      = train['TXT'].values
col_train      = train['COL'].values
cnt_train      = train['CNT'].values

features_val   = preprocess_text(val[['QSN', 'descriptions', 'tags', 'dominant_colors',
                                      'handwritten_text', 'ocr_text']])
txt_val        = val['TXT'].values.astype('float32')
col_val        = val['COL'].values.astype('float32')
cnt_val        = val['CNT'].values.astype('float32')

Training: (3230, 13)
Validation: (513, 13)


In [6]:
# Combining Vizwiz and VQA (create X and Y)

train = pd.concat([vizwiz_train, vqa_train], axis=0)
val   = pd.concat([vizwiz_val, vqa_val], axis=0)
print("Training: {}\nValidation: {}".format(train.shape, val.shape))

features_train = preprocess_text(train[['QSN','descriptions', 'tags', 'dominant_colors', 
                                        'handwritten_text', 'ocr_text']])
txt_train      = train['TXT'].values
col_train      = train['COL'].values
cnt_train      = train['CNT'].values

features_val   = preprocess_text(val[['QSN', 'descriptions', 'tags', 'dominant_colors',
                                      'handwritten_text', 'ocr_text']])
txt_val        = val['TXT'].values.astype('float32')
col_val        = val['COL'].values.astype('float32')
cnt_val        = val['CNT'].values.astype('float32')

Training: (17487, 13)
Validation: (2760, 13)


In [22]:
features_train[random.randint(0,len(features_val))]

'what kind of pudding is this   a person holding a remote control   person indoor remote sitting holding man orange control red television boy girl young bed playing food table room video game white standing shirt  black      '

In [51]:
features_val[random.randint(0,len(features_val))]

'i know this is healthy choice but what dinner is it   a close up of food on a table   indoor table food sitting green plate restaurant wooden sandwich white  black brown  healthy choice complete meall with dessert solwin healthy  complete meal healthy with dessert •hoice 9g fiber 310 calorie homestyle bury salis steak salisbury steak in sautéed onion red apple ca crisp multigrain  '

In [10]:
# remove stop words
stop_words     = set(stopwords.words('english'))
features_train = [w for w in features_train if not w in stop_words]
features_val   = [w for w in features_val if not w in stop_words]

# tokenize
tok        = Tokenizer(num_words=VOCAB_SIZE, 
                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                       lower=True,
                       split=" ")
tok.fit_on_texts(features_train)

# create sequences & pad
train_seq  = tok.texts_to_sequences(features_train)
train_seq  = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq    = tok.texts_to_sequences(features_val)
val_seq    = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

In [11]:
# check class distribution
text_recognition_labels = to_categorical(np.asarray(txt_train)).astype('float32')
color_recognition_labels = to_categorical(np.asarray(col_train)).astype('float32')
print('Number of samples each class: ')
print('Text recognition', text_recognition_labels.sum(axis=0))
print('Color recognition', color_recognition_labels.sum(axis=0))
n_classes = 2

Number of samples each class: 
Text recognition [6247. 8010.]
Color recognition [8844. 5413.]


## LSTM model (skip-gram word2vec)

In [12]:
import numpy as np
import pandas as pd
import os
import pickle
import requests
import time
import gzip

os.environ['KERAS_BACKEND'] = 'tensorflow'

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

import nltk 
import gensim
import logging

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

In [13]:
# punkt sentence level tokenizer
sent_lst = []
for doc in features_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)

In [14]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst,
                                        min_count=6,
                                        size=EMBEDDING_DIM,
                                        sg=1,
                                        workers=os.cpu_count())

2019-03-05 22:25:52,060 : INFO : collecting all words and their counts
2019-03-05 22:25:52,061 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-05 22:25:52,124 : INFO : PROGRESS: at sentence #10000, processed 383294 words, keeping 24880 word types
2019-03-05 22:25:52,151 : INFO : collected 31762 word types from a corpus of 541792 raw words and 14257 sentences
2019-03-05 22:25:52,152 : INFO : Loading a fresh vocabulary
2019-03-05 22:25:52,172 : INFO : effective_min_count=6 retains 3565 unique words (11% of original 31762, drops 28197)
2019-03-05 22:25:52,172 : INFO : effective_min_count=6 leaves 501856 word corpus (92% of original 541792, drops 39936)
2019-03-05 22:25:52,182 : INFO : deleting the raw counts dictionary of 31762 items
2019-03-05 22:25:52,183 : INFO : sample=0.001 downsamples 67 most-common words
2019-03-05 22:25:52,184 : INFO : downsampling leaves estimated 318802 word corpus (63.5% of prior 501856)
2019-03-05 22:25:52,191 : INFO : estima

In [15]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors' % len(embeddings_index))

# Initial word embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 3565 word vectors


In [16]:
def lstm_create_train(labels, learning_rate, lstm_dim, batch_size, num_epochs, optimizer_param, regularization=1e-7):
    
    l2_reg = regularizers.l2(regularization)
    
    # init model
    embedding_layer = Embedding(VOCAB_SIZE,
                                EMBEDDING_DIM,
                                input_length=MAX_DOC_LEN,
                                trainable=True,
                                mask_zero=False,
                                embeddings_regularizer=l2_reg,
                                weights=[embedding_matrix])
    lstm_layer = LSTM(units=lstm_dim, kernel_regularizer=l2_reg)
    dense_layer = Dense(n_classes,
                        activation='softmax', 
                        kernel_regularizer=l2_reg)

    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(lstm_layer))
    model.add(Dropout(0.5))
    model.add(dense_layer)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer_param,
                  metrics=['acc'])
    history = History()
    csv_logger = CSVLogger('./LSTM/text/{}_{}_{}_{}.log'.format(learning_rate, regularization, batch_size, num_epochs),
                           separator=',',
                           append=True)
    t1 = time.time()
    # model fit
    model.fit(train_seq,
              labels.astype('float32'),
              batch_size=batch_size,
              epochs=num_epochs,
              callbacks=[history, csv_logger],
              verbose=2)
    t2 = time.time()
    # save hdf5
    model.save('./LSTM/text/{}_{}_{}_{}_model.h5'.format(learning_rate, regularization, batch_size, num_epochs))
    np.savetxt('./LSTM/text/{}_{}_{}_{}_time.txt'.format(learning_rate, regularization, batch_size, num_epochs), 
               [regularization, (t2-t1) / 3600])
    with open('./LSTM/text/{}_{}_{}_{}_history.txt'.format(learning_rate, regularization, batch_size, num_epochs), "w") as res_file:
        res_file.write(str(history.history))

### Combining Vizwiz + VQA

In [78]:
L = 2e-1
R = 1e-14
B = 32
E = 30
lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=150, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))

Epoch 1/30
 - 31s - loss: 0.4646 - acc: 0.7892
Epoch 2/30
 - 31s - loss: 0.4124 - acc: 0.8254
Epoch 3/30
 - 31s - loss: 0.3873 - acc: 0.8391
Epoch 4/30
 - 31s - loss: 0.3728 - acc: 0.8457
Epoch 5/30
 - 31s - loss: 0.3622 - acc: 0.8525
Epoch 6/30
 - 31s - loss: 0.3563 - acc: 0.8542
Epoch 7/30
 - 31s - loss: 0.3511 - acc: 0.8585
Epoch 8/30
 - 31s - loss: 0.3448 - acc: 0.8593
Epoch 9/30
 - 31s - loss: 0.3416 - acc: 0.8619
Epoch 10/30
 - 31s - loss: 0.3355 - acc: 0.8674
Epoch 11/30
 - 31s - loss: 0.3305 - acc: 0.8671
Epoch 12/30
 - 31s - loss: 0.3310 - acc: 0.8672
Epoch 13/30
 - 31s - loss: 0.3264 - acc: 0.8707
Epoch 14/30
 - 31s - loss: 0.3200 - acc: 0.8730
Epoch 15/30
 - 31s - loss: 0.3205 - acc: 0.8727
Epoch 16/30
 - 31s - loss: 0.3173 - acc: 0.8753
Epoch 17/30
 - 31s - loss: 0.3141 - acc: 0.8760
Epoch 18/30
 - 31s - loss: 0.3098 - acc: 0.8788
Epoch 19/30
 - 31s - loss: 0.3080 - acc: 0.8775
Epoch 20/30
 - 31s - loss: 0.3061 - acc: 0.8792
Epoch 21/30
 - 31s - loss: 0.2993 - acc: 0.8809
E

In [17]:
# todo
L = 2e-1
R = 0
B = 32
E = 30
lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=200, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))

Epoch 1/30
 - 57s - loss: 0.4622 - acc: 0.7857
Epoch 2/30
 - 56s - loss: 0.4101 - acc: 0.8246
Epoch 3/30
 - 55s - loss: 0.3879 - acc: 0.8401
Epoch 4/30
 - 55s - loss: 0.3718 - acc: 0.8455
Epoch 5/30
 - 55s - loss: 0.3643 - acc: 0.8506
Epoch 6/30
 - 56s - loss: 0.3537 - acc: 0.8573
Epoch 7/30
 - 55s - loss: 0.3517 - acc: 0.8567
Epoch 8/30
 - 55s - loss: 0.3442 - acc: 0.8612
Epoch 9/30
 - 55s - loss: 0.3401 - acc: 0.8641
Epoch 10/30
 - 55s - loss: 0.3363 - acc: 0.8658
Epoch 11/30
 - 55s - loss: 0.3346 - acc: 0.8656
Epoch 12/30
 - 55s - loss: 0.3307 - acc: 0.8669
Epoch 13/30
 - 55s - loss: 0.3275 - acc: 0.8704
Epoch 14/30
 - 55s - loss: 0.3216 - acc: 0.8717
Epoch 15/30
 - 55s - loss: 0.3189 - acc: 0.8728
Epoch 16/30
 - 55s - loss: 0.3161 - acc: 0.8744
Epoch 17/30
 - 55s - loss: 0.3161 - acc: 0.8721
Epoch 18/30
 - 55s - loss: 0.3116 - acc: 0.8768
Epoch 19/30
 - 55s - loss: 0.3083 - acc: 0.8791
Epoch 20/30
 - 55s - loss: 0.3056 - acc: 0.8802
Epoch 21/30
 - 57s - loss: 0.3023 - acc: 0.8831
E

## Training and validation on VizWiz only

### VizWiz only - varying learning rate

In [ ]:
# GPU 
L = 1e-8
R = 0
B = 32
E = 30
start = time.time()

lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=150, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

Epoch 1/30
 - 46s - loss: 0.7091 - acc: 0.4469
Epoch 2/30
 - 45s - loss: 0.7090 - acc: 0.4468
Epoch 3/30
 - 45s - loss: 0.7102 - acc: 0.4399
Epoch 4/30
 - 45s - loss: 0.7096 - acc: 0.4419
Epoch 5/30
 - 45s - loss: 0.7090 - acc: 0.4497


In [ ]:
# GPU 
L = 1e-5
R = 0
B = 32
E = 30
start = time.time()

lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=150, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

In [ ]:
# GPU 
L = 1e-3
R = 0
B = 32
E = 30
start = time.time()

lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=150, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

In [ ]:
# GPU 
L = 1e-1
R = 0
B = 32
E = 30
start = time.time()

lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=150, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

### VQA only

In [43]:
# GPU 
L = 1e-1
R = 0
B = 32
E = 30
start = time.time()

lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=100, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

Epoch 1/30
 - 24s - loss: 0.3398 - acc: 0.8830
Epoch 2/30
 - 21s - loss: 0.3161 - acc: 0.8879
Epoch 3/30
 - 20s - loss: 0.3074 - acc: 0.8904
Epoch 4/30
 - 21s - loss: 0.2982 - acc: 0.8895
Epoch 5/30
 - 21s - loss: 0.2935 - acc: 0.8938
Epoch 6/30
 - 20s - loss: 0.2887 - acc: 0.8910
Epoch 7/30
 - 21s - loss: 0.2899 - acc: 0.8938
Epoch 8/30
 - 21s - loss: 0.2824 - acc: 0.8929
Epoch 9/30
 - 21s - loss: 0.2793 - acc: 0.8966
Epoch 10/30
 - 21s - loss: 0.2823 - acc: 0.9000
Epoch 11/30
 - 21s - loss: 0.2806 - acc: 0.8972
Epoch 12/30
 - 21s - loss: 0.2748 - acc: 0.9009
Epoch 13/30
 - 21s - loss: 0.2723 - acc: 0.8985
Epoch 14/30
 - 21s - loss: 0.2731 - acc: 0.9000
Epoch 15/30
 - 21s - loss: 0.2683 - acc: 0.9034
Epoch 16/30
 - 20s - loss: 0.2660 - acc: 0.9031
Epoch 17/30
 - 20s - loss: 0.2660 - acc: 0.9037
Epoch 18/30
 - 21s - loss: 0.2584 - acc: 0.9074
Epoch 19/30
 - 21s - loss: 0.2643 - acc: 0.9025
Epoch 20/30
 - 21s - loss: 0.2643 - acc: 0.9102
Epoch 21/30
 - 21s - loss: 0.2597 - acc: 0.9077
E

In [44]:
# GPU 
L = 1e-2
R = 0
B = 32
E = 30
start = time.time()

lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=100, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

Epoch 1/30
 - 25s - loss: 0.4166 - acc: 0.8703
Epoch 2/30
 - 21s - loss: 0.3435 - acc: 0.8892
Epoch 3/30
 - 21s - loss: 0.3371 - acc: 0.8892
Epoch 4/30
 - 21s - loss: 0.3350 - acc: 0.8892
Epoch 5/30
 - 21s - loss: 0.3348 - acc: 0.8892
Epoch 6/30
 - 21s - loss: 0.3298 - acc: 0.8892
Epoch 7/30
 - 21s - loss: 0.3309 - acc: 0.8892
Epoch 8/30
 - 21s - loss: 0.3281 - acc: 0.8892
Epoch 9/30
 - 21s - loss: 0.3237 - acc: 0.8892
Epoch 10/30
 - 21s - loss: 0.3200 - acc: 0.8898
Epoch 11/30
 - 21s - loss: 0.3226 - acc: 0.8895
Epoch 12/30
 - 21s - loss: 0.3167 - acc: 0.8901
Epoch 13/30
 - 21s - loss: 0.3181 - acc: 0.8904
Epoch 14/30
 - 21s - loss: 0.3163 - acc: 0.8901
Epoch 15/30
 - 21s - loss: 0.3130 - acc: 0.8895
Epoch 16/30
 - 21s - loss: 0.3127 - acc: 0.8898
Epoch 17/30
 - 21s - loss: 0.3116 - acc: 0.8901
Epoch 18/30
 - 21s - loss: 0.3063 - acc: 0.8907
Epoch 19/30
 - 21s - loss: 0.3059 - acc: 0.8889
Epoch 20/30
 - 21s - loss: 0.3064 - acc: 0.8898
Epoch 21/30
 - 21s - loss: 0.3034 - acc: 0.8913
E